In [1]:
!pip install --upgrade typing_extensions

In [2]:
!pip install vegafusion

In [3]:
!pip install "vl-convert-python>=1.6.0"

In [4]:
import pandas as pd
import altair as alt
alt.data_transformers.enable('vegafusion')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


DataTransformerRegistry.enable('vegafusion')

In [7]:
import pandas as pd
import altair as alt

# Disable row limit
alt.data_transformers.disable_max_rows()

df = pd.read_csv('movies_cleaned.csv')
df['primary_genre'] = df['genres'].str.split(',').str[0].str.strip()
df = df[df['popularity'] < 900]

# Converting to dictionary for VegaFusion compatibility
data_values = df.to_dict(orient = 'records')
data = alt.InlineData(values = data_values)

# Creating dropdown parameter for genre
genre_dropdown = alt.param(
    name = 'SelectedGenre',
    bind = alt.binding_select(
        options = sorted(df['primary_genre'].dropna().unique()),
        name = 'Genre:'
    ),
    value = 'Action'
)

# Chart with points and opacity
points = alt.Chart(data).mark_point(filled = True, opacity = 0.5).encode(
    x = alt.X('popularity:Q', title = 'Popularity'),
    y = alt.Y('vote_average:Q', title = 'Average Rating'),
    color = alt.condition(
        'datum.primary_genre == SelectedGenre',
        alt.Color('primary_genre:N', legend = None, title = 'Highlighted Genre'),
        alt.value('lightgray')
    ),
    tooltip = ['title:N', 'popularity:Q', 'vote_average:Q', 'primary_genre:N']
)

# Defining chart with no shape encoding
chart = alt.Chart(data).mark_point(filled = True).encode(
    x = alt.X('popularity:Q', title = 'Popularity'),
    y = alt.Y('vote_average:Q', title = 'Average Rating'),
    color = alt.condition(
        'datum.primary_genre == SelectedGenre',
        alt.Color('primary_genre:N', legend = None, title = 'Highlighted Genre'),
        alt.value('lightgray')
    ),
    tooltip = ['title:N', 'popularity:Q', 'vote_average:Q', 'primary_genre:N']
).add_params(
    genre_dropdown
).properties(
    title = 'Movie Popularity vs. User Rating by Genre',
    width = 700,
    height = 500
).interactive()

chart
chart.save('movie_chart.html')